In [11]:
%load_ext autoreload
%autoreload 2
import os
import glob
import random
from pprint import pprint
from iucrime.records import Record

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
path = '../../data/text'
files = glob.glob(os.path.join(path, '*'))

Let's just pick one data file and take a look

In [22]:
fname = '../../data/text/2013/1-1-13.txt'
r = Record(open(fname).read())
pprint(r.info)

{'disposition': 'closed by arrest',
 'event number': '13-01-01-000047',
 'from datetime': datetime.datetime(2013, 1, 1, 23, 45),
 'incident': 'all other offenses - trespass & malicious trespass',
 'location': 'indiana memorial union',
 'report number': '130002',
 'to datetime': datetime.datetime(2013, 1, 1, 23, 52)}


In [8]:
print(open(fname).read())

Indiana University, Bloomington
Police Department
Student Right To Know CAD Daily Log

From Jan 01, 2013 to Jan 01, 2013.
Date Reported: 01/01/13 - TUE at 23:49
Location : INDIANA MEMORIAL UNION
Date and Time Occurred From - Occurred To
01/01/13 - TUE at 23:45 - 01/01/13 - TUE at 23:52
Incident : ALL OTHER OFFENSES - TRESPASS & MALICIOUS TRESPASS
Disposition: CLOSED BY ARREST

Event #: 13-01-01-000047
Report #: 130002

1 Incidents Listed.

Print Date and Time

1/2/2013

6:02:05PM

at Page No.

1


